<a href="https://colab.research.google.com/github/ntolayd/CE888/blob/main/Lab4/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import pandas as pd
import numpy as np

In [111]:
data = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv',header=None)


In [112]:
data = data.replace(99, np.nan)

In [113]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
count,24902.000000,16452.000000,18184.000000,16642.000000,15608.000000,24977.000000,20431.000000,24976.000000,24981.000000,15752.000000,20324.000000,21821.000000,22423.000000,24981.000000,22515.000000,24977.000000,24975.000000,24980.000000,24979.000000,24978.000000,24980.000000,24921.000000,21433.000000,20033.000000,15879.000000,20816.000000,23750.000000,24904.000000,23990.000000,24934.000000,17977.000000,24656.000000,24948.000000,16704.000000,21563.000000,24952.000000,24965.00000,16881.000000,22858.000000,23080.000000,...,24805.000000,24936.000000,20164.000000,17276.000000,24728.000000,24923.000000,17493.000000,24940.000000,24916.000000,20222.000000,8532.000000,8693.000000,8616.000000,8761.000000,8793.000000,8860.000000,8944.000000,9021.000000,9074.000000,9024.000000,9223.000000,9124.000000,9286.000000,9297.000000,9368.000000,9469.000000,9452.000000,9604.000000,9564.000000,9728.000000,9672.000000,9889.000000,9988.000000,10200.000000,10265.000000,10370.000000,10520.000000,10666.000000,10910.000000,9943.000000
mean,72.381174,0.904573,0.208460,0.316574,-1.449114,0.403556,1.594447,-0.429662,-0.624085,-0.557031,1.301231,1.805797,1.449290,-1.762882,1.396334,-1.706201,-3.104818,-1.115141,-0.635103,0.154227,-0.933997,2.095626,0.868662,0.109192,-1.687420,0.382373,1.286232,3.189821,1.518830,2.972455,-0.436493,2.198943,3.155988,-1.368648,0.859494,3.006351,3.31250,-1.363987,1.300479,1.067337,...,2.459180,2.976021,0.246998,-0.667938,2.273820,2.541538,-0.863868,2.639132,2.559463,0.418405,-0.687494,2.757389,1.082492,-1.553548,-0.270715,2.520047,0.715392,1.751897,0.091667,1.122074,1.836562,0.902718,2.028400,0.694418,0.918927,0.344681,1.900667,2.109940,3.574720,0.612016,2.038527,1.179744,2.504041,1.173482,1.039130,1.528003,1.673750,0.767692,-0.031244,1.354711
std,21.842598,5.171798,5.597327,5.399806,5.234654,5.255700,4.900540,5.495931,4.929144,5.162224,5.087359,4.916611,4.847879,5.076765,5.122385,5.123290,4.956258,4.398482,4.879306,4.887486,5.100931,4.881075,5.099855,5.110971,5.229222,5.044315,4.856339,4.602144,4.977191,4.792813,4.936876,4.916673,4.456629,5.248554,5.034648,4.597890,4.28162,5.200415,5.011207,4.867559,...,4.718655,4.547434,5.143053,5.152378,5.120954,4.701836,5.216555,4.621860,4.662009,4.771693,5.808941,4.529657,5.224221,5.032757,5.427726,4.724249,5.143225,4.932234,5.307042,5.523134,5.338919,5.054666,5.104706,5.118793,5.290301,5.133460,4.782956,4.703183,4.799183,5.259641,5.060445,5.127291,4.611339,5.140307,5.167043,4.852069,5.025076,5.393833,5.176806,5.300409
min,36.000000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.95000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,53.000000,-2.670000,-4.420000,-4.030000,-5.870000,-3.690000,-1.310000,-5.290000,-4.610000,-4.760000,-2.090000,-1.210000,-1.650000,-6.260000,-2.090000,-6.210000,-7.430000,-4.420000,-4.420000,-3.450000,-5.050000,-0.780000,-2.820000,-3.790000,-6.210000,-3.350000,-1.840000,0.680000,-1.650000,0.340000,-4.170000,-0.630000,0.630000,-5.920000,-2.770000,0.390000,0.97000,-5.870000,-2.140000,-2.140000,...,-0.290000,0.440000,-3.640000,-4.810000,-0.780000,-0.2

In [114]:
data = data.drop([0], axis=1)

In [132]:
data.shape

(24983, 100)

In [134]:
sorted(data.mean())

[-3.92072789781688,
 -3.104362765315398,
 -2.386882492177367,
 -2.2826727421248414,
 -1.8856804441959503,
 -1.7631531589666969,
 -1.7151830259888905,
 -1.70566664245122,
 -1.627823235442717,
 -1.5569490504010834,
 -1.5435742443674165,
 -1.1149414914189595,
 -1.0817885995174172,
 -1.0215950124530346,
 -0.9336547041528531,
 -0.9260496670656038,
 -0.8798091885400717,
 -0.8620786426186259,
 -0.7627974855477999,
 -0.7576744854326976,
 -0.6347126836911786,
 -0.6240000325716979,
 -0.6234831560885146,
 -0.6051536654184694,
 -0.6025274238051392,
 -0.48585891573930107,
 -0.42882660886627344,
 -0.22954087125595496,
 -0.13860853378044283,
 -0.06946891494305332,
 0.01694054443928221,
 0.09860422768995264,
 0.12270591625476543,
 0.14725799298483627,
 0.15438907653607375,
 0.2793674215259832,
 0.3523769391842546,
 0.3623557576800315,
 0.39779460188976223,
 0.4034934564521284,
 0.4184427934849018,
 0.44874857925073597,
 0.5286579895028575,
 0.6064156451416435,
 0.6236936996484771,
 0.7401507234297903,

In [139]:
data.iloc[16779,79]

-8.54

In [115]:
arr = data.values

In [116]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """

  new_data = orig.copy()
  rated = np.where(~np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [117]:
user_ratings,idx = replace(arr)

In [118]:
user_ratings

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [124]:
n_latent_factors = 4

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [120]:
latent_user_preferences

array([[0.61297817, 0.96602266, 0.56113922, 0.78852887],
       [0.64914282, 0.33161091, 0.26393971, 0.94892024],
       [0.29951754, 0.35036078, 0.87745869, 0.20761362],
       ...,
       [0.21969979, 0.70520465, 0.24856712, 0.83178411],
       [0.9663157 , 0.05728444, 0.81730033, 0.71508874],
       [0.1264497 , 0.2407062 , 0.67834983, 0.95308108]])

In [121]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if (rating != 99 and not np.isnan(rating)):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
        mse_history.append(mse)
    return mse_history


In [125]:
sgd(100)

Iteration 0/100:	MSE=25.698012
Iteration 1/100:	MSE=25.016531
Iteration 2/100:	MSE=24.886117
Iteration 3/100:	MSE=24.772425
Iteration 4/100:	MSE=24.658887
Iteration 5/100:	MSE=24.544041
Iteration 6/100:	MSE=24.427570
Iteration 7/100:	MSE=24.309295
Iteration 8/100:	MSE=24.189012
Iteration 9/100:	MSE=24.066412
Iteration 10/100:	MSE=23.941004
Iteration 11/100:	MSE=23.812020
Iteration 12/100:	MSE=23.678288
Iteration 13/100:	MSE=23.538070
Iteration 14/100:	MSE=23.388856
Iteration 15/100:	MSE=23.227127
Iteration 16/100:	MSE=23.048108
Iteration 17/100:	MSE=22.845596
Iteration 18/100:	MSE=22.611991
Iteration 19/100:	MSE=22.338802
Iteration 20/100:	MSE=22.017938
Iteration 21/100:	MSE=21.644058
Iteration 22/100:	MSE=21.217865
Iteration 23/100:	MSE=20.749317
Iteration 24/100:	MSE=20.258790
Iteration 25/100:	MSE=19.774207
Iteration 26/100:	MSE=19.324166
Iteration 27/100:	MSE=18.930168
Iteration 28/100:	MSE=18.602111
Iteration 29/100:	MSE=18.338843
Iteration 30/100:	MSE=18.132136
Iteration 31/100:	

[25.69801200906223,
 25.016531477207646,
 24.886116876428073,
 24.77242509809163,
 24.658886744412854,
 24.54404108071947,
 24.427569886519805,
 24.309295208457716,
 24.189012188026897,
 24.06641177510353,
 23.941003659674127,
 23.81201973509342,
 23.67828814577233,
 23.538070199554937,
 23.388856258369767,
 23.22712688811736,
 23.048108346116074,
 22.84559569729933,
 22.611990815512335,
 22.338802441839523,
 22.017937877717536,
 21.644057982153434,
 21.217864501231436,
 20.749316523083273,
 20.25879016628834,
 19.77420668803801,
 19.3241655064676,
 18.930168055220374,
 18.60211051053299,
 18.338842980363946,
 18.132135971030255,
 17.971099113957568,
 17.845202164075577,
 17.745642939215415,
 17.66561069676459,
 17.60004212409241,
 17.545239654340868,
 17.49851541961994,
 17.457908807377922,
 17.42197753828955,
 17.38964793852701,
 17.36010905625745,
 17.332738106864007,
 17.30704797380117,
 17.282650162212533,
 17.259228581964578,
 17.236520944052103,
 17.21430553748032,
 17.192391835

In [23]:
aidx

(array([13196,   272,  1438, ...,  7591,  2764, 12046]),
 array([83, 36, 65, ..., 64, 76,  5]))

In [127]:
#prediction for validation set
error_list = []
for i,j in zip(idx[0],idx[1]):
  error_list.append(predict_rating(i,j) - arr[i,j])

final_mse = (np.array(error_list)**2).mean()
print(final_mse)

17.294897620781708


In [130]:
#prediction for nans
np.where(np.isnan(arr))

for i,j in zip(np.where(np.isnan(arr))[0], np.where(np.isnan(arr))[1]):
  arr[i,j] = predict_rating(i,j)

In [129]:
np.where(np.isnan(arr))

array([    0,     0,     0, ..., 24982, 24982, 24982])

In [140]:
## movies dataset

movie_items_data = pd.read_excel('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx', sheet_name='Items')
movie_users_data = pd.read_excel('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx', sheet_name='Users')

In [162]:
movie_users_data

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
5,783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
7,768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
8,4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983
9,1882,-0.083093,-0.029160,0.013748,0.022716,-0.062732,-0.080670,-0.019723,0.013253,-0.091305,-0.090687,0.030638,-0.113364,-0.021325,0.181508,0.022357


In [147]:
movie_items_data.sort_values(by='Factor1', ascending=False)

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
86,4327,Charlie's Angels (2000),4.023815,0.882963,1.138274,-0.431058,0.208086,-0.853887,-0.590821,0.416106,0.387390,0.522155,-0.379812,-0.567725,-1.007534,0.794589,-2.247246
37,414,Batman Forever (1995),3.111997,-0.102569,0.514301,-0.720549,0.773486,-0.092841,-1.040723,-0.308222,-1.720265,-0.769622,-0.543568,-0.082656,0.343975,-0.760857,-1.959942
85,3049,Ace Ventura: Pet Detective (1994),2.723720,0.745282,0.029587,-0.328153,0.685947,2.592889,3.457525,-0.780380,-0.219994,-1.313421,0.163827,0.573048,-0.324509,0.358344,0.128814
90,8467,Dumb & Dumber (1994),2.720287,1.217620,-0.101601,-0.119204,1.279939,2.805066,3.524783,-1.283701,-0.133343,-1.343301,-1.009496,1.808762,-1.240696,0.279910,1.616338
68,854,The Mask (1994),2.263142,-0.067761,-0.205463,0.068880,-0.189090,2.078554,2.444922,-0.075971,-0.674767,-0.198560,1.641380,-0.839276,-0.517086,-1.058431,-0.931746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,122,The Lord of the Rings: The Return of the King ...,-2.271110,-2.526300,2.142566,0.341954,2.688624,-0.360610,1.140343,-0.932931,1.356982,1.331742,-0.454221,-0.620533,-0.638745,-0.172093,-0.314937
59,680,Pulp Fiction (1994),-2.382739,2.085395,0.683164,-0.841393,-0.062049,0.326063,0.624981,1.071625,-0.037820,-0.729796,0.412544,-0.033415,0.062671,0.228910,-1.833900
27,238,The Godfather (1972),-2.611462,0.005551,-1.055331,0.150950,0.233674,0.266476,0.605631,0.595702,-0.112474,-1.488997,-0.378031,0.303310,-0.591693,0.400705,-2.267831
33,278,The Shawshank Redemption (1994),-2.780308,-0.088747,-1.821590,-0.157495,0.323285,-0.114768,0.281397,-0.130262,0.028689,-0.810090,-0.229137,0.587575,0.194144,-0.976664,-0.779991


In [161]:
np.array(movie_users_data[movie_users_data['User'] == 2848].loc[:,'Factor1':]).dot(np.array(movie_items_data[movie_items_data['Movie ID']==558].loc[:,'Factor1':]).reshape(-1,1))

array([[0.13261348]])

In [163]:
movie_users_data[movie_users_data['User'] == 3878]

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.18925,0.06599,0.102776


In [165]:
np.array(movie_items_data[movie_items_data['Movie ID']==558].loc[:,'Title':])

array([['Spider-Man 2 (2004)', 2.2352233063746585, -0.7881294761636518,
        0.5967221341575716, -0.40008219852561283, -1.246622711078014,
        -1.507203113683306, -0.21991090147438985, -0.4862538356609814,
        0.3316574543781727, -1.8337552665430414, -2.243340340044728,
        -1.1979378607869537, -1.4508122479345416, -2.381119005004064,
        -0.4557528272511082]], dtype=object)

In [169]:
movie_items_data.drop(['Movie ID'],axis=1).values[0][1:]

array([-1.5218476932977845, -1.0385074424076217, 2.0272691992004397,
       0.24793252416549907, -0.5945476708705433, 2.5132599646492184,
       -1.8489100782588614, 0.47670996550851713, -0.22414630395868232,
       -0.7606806322851536, -0.973915043301705, 0.8623788058062587,
       0.4038611510962773, 1.129616009775629, -0.24880587477821164],
      dtype=object)

In [195]:
def recommender(user_id):
  user_array = np.array(movie_users_data[movie_users_data['User'] == user_id].loc[:,'Factor1':])
  df_scores = pd.DataFrame(columns=['ratings','Movie'])
  for i in movie_items_data.drop(['Movie ID'],axis=1).values:
    rating = user_array.dot(i[1:].reshape(-1,1))
    movie_dict = {'ratings':rating, 'Movie':i[0]}
    df_scores = df_scores.append(movie_dict, ignore_index=True)
  return df_scores.sort_values('ratings', ascending=False).head(3)


In [198]:
recommender(2848)

,ratings,Movie
21,[[1.007056895264574]],The Dark Knight (2008)
16,[[0.8444536723338026]],The Lord of the Rings: The Return of the King ...
14,[[0.7882439990139722]],The Lord of the Rings: The Fellowship of the R...
